## Necessary Dependencies

In [ ]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox 
import time
from tkinter.ttk import *
from tkinter import *
import math
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from collections import deque 
import tensorflow as tf
import pandas as pd

## Naive Bayes Classifier

In [ ]:
def Naive_Bayes_Classifier(featureList, X, y):
    global m_estimate_flag
    print('m estimate', m_estimate_flag)
    def TrainingSamplesOfClass(classID, out_class):
        # count classID and return
        temp=0
        for i in out_class:
            if i==classID:
                temp= temp+1
        return temp
    
    def calculatePriorProbability(count, length):
        return count/length

    classes = np.unique(y.values)
    target = dict()
    for label in range(len(classes)):
        target[classes[label]]=label;
    """
    key: classes[i], value: target[classes[i]]
    Iris-setosa: 0
    """
    # each class counting/total in total samples
    count_class = list()
    for i in range(len(classes)):
        count_class.append(TrainingSamplesOfClass(classes[i],y.values))

    # CALCULATING PRIOR PROBABILITIES
    prior_class = list()
    for i in range(len(classes)):
        prior_class.append(calculatePriorProbability(count_class[i],len(y.values)))

    def featureCountGivenThatLabel(f_val_x, features, classID, out_class):
        count=0
        for fx, o_class in zip(features, out_class):
                if fx == f_val_x and o_class == classID:
                    count = count+1
        return count

    def calculateProbability(class_i, like_class):
        sum_of_likelihoods = 0
        for lik in like_class:
            sum_of_likelihoods = sum_of_likelihoods + lik
        import sys
        if sum_of_likelihoods == 0:
            print('No existence found for any of your entered feature so use m-estimate')
            global root
            root.destroy()
            sys.exit()
        return like_class[target[class_i]]/sum_of_likelihoods

    def calculateLikelihood(condProbility_featureList_Class, prior_Class):
        product =1;
        for p in condProbility_featureList_Class:
            product = product*p;
        return product * prior_Class

    def m_estimate(a, b, p, m):
        return (a + m*p)/(b + m)

    def NBC():
        no_of_features = len(featureList)

        # CALCULATING CONDITIONAL PROBABILITIES
        # for N number of Classes
        conditional_counts = list()
        conditional_probabilities = list()

        for class_i in range(len(classes)):
            count_list = []
            prob_list = []
            for feature in range(len(featureList)):
                count_list.append(featureCountGivenThatLabel(featureList[feature], X[X.columns[feature]].values, 
                                                             classes[class_i], y.values))
                prob_list.append(count_list[feature]/count_class[class_i])
                if m_estimate_flag == 1 and prob_list[-1] == 0:
                    t = len(np.unique(X[X.columns[feature]].values))
                    prob_list[-1] = m_estimate(count_list[-1], count_class[class_i], 1/t, 
                                               len(X[X.columns[feature]].values))
            conditional_counts.append(count_list)
            conditional_probabilities.append(prob_list)

        # CALCULATING LIKELIHOODS
        like_class = list()
        for i in range(len(classes)):
            like_class.append(calculateLikelihood(conditional_probabilities[i] , prior_class[i]))

        # CALCULATING POSTERIOR PROBABILITIES
        prob_class = []
        maxim=0

        for i in range(len(classes)):
            lst = []
            lst.append(calculateProbability(classes[i], like_class))
            if maxim < lst[-1]:
                maxim = lst[-1]
            lst.append(classes[i])
            prob_class.append(lst)

        prob_class.sort(key=lambda x:float(x[0]), reverse=True)
        predicted_class = 'Class_i'
        GUI_msg = str()
        for i in range(len(classes)):
            #GUI_msg = GUI_msg + 'Predicted Outcome using NBC is ' + str(prob_class[i][1]) + ' with probability ' + str(prob_class[i][0]*100) + "\n"
            if i==0:
                #print('Predicted Outcome using NBC is',prob_class[i][1],'with probability', prob_class[i][0]*100)#maxim*100)
                GUI_msg = 'Predicted Class using Naive Bayes Classifier is ' + str(prob_class[i][1]) + ' with probability ' + str(round(prob_class[i][0]*100, 2))
                predicted_class = prob_class[i][1]
        return GUI_msg, predicted_class
    return NBC()

## K-NN

In [ ]:
def k_Nearest_Neighbor(featureList, X, y):
    global k, distance_metric_choice
    def calcEuclidean():
        row = list()
        for i in range(len(X)):
            #print(i,' : ')
            t3 = 0
            for j in range(len(X.iloc[0])):
                #print('\t',j)
                # Take (P1-P2)^2 first
                t1 = X.iloc[i][j] - featureList[j] # inner bracket
                t2 = t1 * t1 # Squaring
                t3 = t3 + t2 # Adding all values under Square Root

            row.append(math.sqrt(t3)) # Each entry is distance of corresponding row in X
            #row[i].append(Y.iloc[i])

        row = pd.DataFrame(row)
        dist = pd.concat([row, y], axis=1) # Concating Targets at the end of distance of each sample
        # adding column name to the respective columns 
        dist.columns =['Distance', 'Target'] 
        dist.sort_values(by=['Distance'], inplace=True)  #  Sorting W.R.T Distance Values    
        distK = dist.head(k)  #  Extracting only K number of values
        #display(distK)
        result = distK['Target'].mode() # Taking Target with maximum occurrences
        print('Result :\n',result[0]) # Displaying only 1st Output
        #print('Result :\n',result) # Displaying only 1st Output
        predicted_class = 'Class_i'
        GUI_msg = str()
        GUI_msg = 'Predicted Class using k-Nearest Neighbors is ' + str(result[0])
        predicted_class = str(result[0])
        return GUI_msg, predicted_class

    def calcManhattan():
        row = list()
        for i in range(len(X)):
            #print(i,' : ')
            t3 = 0
            for j in range(len(X.iloc[0])):
                #print('\t',j)

                t1 = X.iloc[i][j] - featureList[j] # inner bracket --> (P1-P2)^2
                t2 = t1 * t1 # Squaring
                t3 = t3 + t2 # Adding all values under Square Root
            row.append(t3) # Each entry is distance of corresponding row in X

        row = pd.DataFrame(row)
        dist = pd.concat([row, y], axis=1) # Concating Targets at the end of distance of each sample
        dist.columns =['Distance', 'Target']  # adding column name to the respective columns 
        dist.sort_values(by=['Distance'], inplace=True)  #  Sorting W.R.T Distance Values
        distK = dist.head(k)  #  Extracting only K number of values
        #display(distK)
        result = distK['Target'].mode() # Taking Target with maximum occurrences
        print('Result :\n',result[0]) # Displaying only 1st Output
        predicted_class = 'Class_i'
        GUI_msg = str()
        GUI_msg = 'Predicted Class using k-Nearest Neighbors is ' + str(result[0])
        predicted_class = str(result[0])
        return GUI_msg, predicted_class
    
    #print(distance_metric_choice, type(distance_metric_choice))
    if distance_metric_choice == 1:  # For Manhattan Distance
        #print('calling manhattan')
        return calcManhattan()
    elif distance_metric_choice == 2:  # For Euclidean Distance
        #print('calling euclidean')
        return calcEuclidean()

## Decision Tree Classifier

In [ ]:
def Decision_Tree_Classifier(featureList, X, y):
    global max_depth
    clf = DecisionTreeClassifier(criterion="entropy", max_depth = max_depth, 
                             random_state = 0)
    X.to_numpy()
    y.to_numpy()
    clf = clf.fit(X, y)
    
    # Predict for 1 observation
    featureList = np.array(featureList)
    featureList = featureList.reshape(1, -1)
    #print('print shape ', featureList.shape)
    y_pred = clf.predict(featureList)
    #print('print prediction ', y_pred)
    predicted_class = y_pred[0]
    GUI_msg = str()
    GUI_msg = 'Predicted Class using Decision Tree Classifier is ' + predicted_class
    return GUI_msg, predicted_class

# GUI

In [ ]:
# GLOBAL VARIABLES--------------------------------------------------------------------------------------------------
classification_count = 0
m_estimate_flag = 0
distance_metric_choice=0
k=0
max_depth=0
stack=[]
load_dataset_count=0
# ------------------------------------------------------------------------------------------------------------------
def rbutton(v):
    stack.append(v.get())
    print(v.get())
    
def take_input(root, X_data, y_data, chosen_algo):
    no_of_features = len(X_data.columns)
    label = Label(root, 
                  text = "Enter Unseen sample in below "+str(no_of_features)+" entry cells and then click on "
                  "Show Classification button",
                  font='Arial 16 bold', fg='white', bg="grey").place(x=100, y=100)
    horizontal=105
    hori_ratio = int((900/no_of_features))
    width_ratio = int(hori_ratio/(no_of_features**2))

    for i in range(no_of_features):
        value = i#'feature %d'%(i+1)
        if width_ratio<10:
            width_ratio = 10
        t= Text(root, height=1, width=width_ratio, font='Arial '+str(width_ratio))
        t.insert('1.0', value)
        t.place(x=horizontal, y=150)
        stack.append(t)
        horizontal = horizontal + hori_ratio
    show_classification = tk.Button(root, text='Show Classification',
                padx=35, pady=10,
                fg="white", bg="grey", command=lambda:classify(root, chosen_algo, X_data, y_data, show_classification))
    show_classification.pack(side=tk.RIGHT)    

count_wrong_k=0 
def setting_parameters_chosen_algo(root, top1, chosen_algo, X_data, y_data):
        
    if chosen_algo == "Naive Bayes Classifier":
        
        Label3 = Label(top1, text="Do you want to use m-estimate technique:", font='Arial 10').place(x=70, y=50)
        var = IntVar(top1)
        Check_Button = Checkbutton(top1, text='M-Estimate',variable=var, onvalue=1, offvalue=0,
                                   command=lambda:rbutton(var)).place(x=120, y=70)
        button1 = Button(top1, text = "Submit Your Choice", padx=20, pady=5, fg="white", bg="grey",
                    command = lambda : submit_naive_parameters(root, top1, X_data, y_data)).pack(side=tk.BOTTOM)#top1.destroy)
        def submit_naive_parameters(root, top1, X_data, y_data):
            global m_estimate_flag
            if len(stack) != 0:
                m_estimate_flag = stack.pop()
                stack.clear()
            top1.destroy()
            take_input(root, X_data, y_data, chosen_algo)
        
    elif chosen_algo == "k-Nearest Neighbor":
        global distance_metric_choice
        Label3 = Label(top1, text="k (1 to "+str(len(X_data))+"):", font='Arial 12').place(x=100, y=40)
        k_var = StringVar(top1)
        entry = Entry(top1, textvariable=k_var).place(x=190, y=43)
        Label4 = Label(top1, text="Distance Metric:", font='Arial 12').place(x=100, y=70)
        v = StringVar(top1, "1")
        distance_metric_choice=1
        rb1 = Radiobutton(top1, text = "Manhattan", variable = v,  
                    value = "1", indicator = 0, command =lambda:rbutton(v)).place(x=220, y=70)
        rb2 = Radiobutton(top1, text = 'Euclidean', variable = v,  
                    value = "2", indicator = 0, command =lambda:rbutton(v)).place(x=300, y=70)
        button1 = Button(top1, text = "Submit Your Choice", padx=20, pady=5, fg="white", bg="grey",
                    command = lambda : submit_knn_parameters(root, top1, k_var.get(), X_data, y_data)).pack(side=tk.BOTTOM)#top1.destroy)
        def submit_knn_parameters(root, top1, k_var, X_data, y_data):
            global k
            global distance_metric_choice, count_wrong_k
            k = int(k_var)
 
            if len(stack) != 0:
                distance_metric_choice = int(stack[-1])
            #print('distance_metric_choice in knn paramter',distance_metric_choice)
                
            if(k < 1 or k > len(X_data)):
                count_wrong_k = count_wrong_k + 1
                if count_wrong_k==1:
                    Label3 = Label(top1, text="Invalid k value, try again", font='Arial 10').place(x=100, y=95)
            else:
                stack.clear()
                top1.destroy()
                take_input(root, X_data, y_data, chosen_algo)
            
            
    elif chosen_algo == "Decision Tree":
        
        Label3 = Label(top1, text="Max Depth:", font='Arial 12').place(x=100, y=70)
        maxDepth_var = StringVar(top1)
        entry = Entry(top1, textvariable=maxDepth_var).place(x=200, y=73)
        button1 = Button(top1, text = "Submit Your Choice", padx=20, pady=5, fg="white", bg="grey",
                    command = lambda : submit_dtree_parameters(root, top1, maxDepth_var.get(), X_data, y_data)).pack(side=tk.BOTTOM)#top1.destroy)
        def submit_dtree_parameters(root, top1, maxDepth_var, X_data, y_data):
            global max_depth
            max_depth = int(maxDepth_var)
                
            top1.destroy()
            take_input(root, X_data, y_data, chosen_algo)

def submit_algo(root, top1, values, X_data, y_data):
    # clear window Toplevel 1
    lst = top1.pack_slaves()
    for l in lst:
        l.destroy()
    # list out keys and values separately
    key_list = list(values.keys())
    val_list = list(values.values())

    # print key with val
    if len(stack) != 0:
        val = str(stack.pop())
        position = val_list.index(val)
        stack.clear()
        chosen_algo = key_list[position]
        label = Label(top1, text = "Chosen Algo is " + chosen_algo, font='Arial 12').pack()   
        setting_parameters_chosen_algo(root, top1, chosen_algo, X_data, y_data)
    


def load_dataset():
    global load_dataset_count
    load_dataset_count = load_dataset_count + 1
    def load_dataset_first_time():
        csv_data=''
        while csv_data=='':
            csv_data = filedialog.askopenfilename(initialdir="/Downloads", title="Choose a Dataset to to load its saved samples",
                                           filetypes=(("all files", "*.*"), ("csv files", "*.csv")))
            if csv_data=='':
                messagebox.showerror("ERROR", 'Unable to find or open file ' + csv_data)
        data = pd.read_csv(csv_data)
        last_column = data.columns.values[len(data.columns)-1]
        # X: features y: labels
        X_data = data.drop(columns=last_column)
        y_data = data[[last_column]]

        #---------------------------------------------------------------------------------------------------------------
        # Messagebox
        messagebox.showinfo("Dataset Load Status", "Chosen Dataset has been loaded") 

        # Creating widget for Algorithm choice 
        top1 = Toplevel(root)
        top1.title("Algorithm Choice Selection")
        top1.geometry("400x200")
        v = StringVar(top1) 

        # Dictionary to create multiple buttons 
        values = {"Naive Bayes Classifier" : "1", 
                  "k-Nearest Neighbor" : "2", 
                  "Decision Tree" : "3"} 
        # Loop is used to create multiple Radiobuttons 
        # rather than creating each button separately 
        for (text, value) in values.items(): 
            Radiobutton(top1, text = text, variable = v,  
                        value = value, indicator = 0, 
                        background = "light blue", command =lambda:rbutton(v)).pack(fill = X, ipady = 5)
        button1 = Button(top1, text = "Submit Your Choice", padx=20, pady=5, fg="white", bg="grey",
                        command = lambda : submit_algo(root, top1, values, X_data, y_data))#top1.destroy)     
        button1.pack(side=tk.BOTTOM)
        top1.mainloop() 
    def load_dataset_not_first_time():
        # Creating widget for Algorithm choice 
        top1 = Toplevel(root)
        top1.title("Dataset Reload Choice Selection")
        top1.geometry("400x200")
        Label3 = Label(top1, text="Do you want to load another dataset?", 
                       font='Arial 14').place(x=40, y=10)
        Label4 = Label(top1, text="Note: Labels should be in the last column of a csv file", 
                       font='Arial 9').place(x=45, y=50)
        button1 = Button(top1, text = "Yes", padx=20, pady=5, fg="white", bg="grey",
                        command = lambda : load_dataset_first_time())
        button1.place(x=120, y=170)
        button2 = Button(top1, text = "No", padx=20, pady=5, fg="white", bg="grey",
                        command = lambda : top1.destroy())
        button2.place(x=210, y=170)
        top1.mainloop() 
        
    if load_dataset_count == 1:
        load_dataset_first_time()
    else:
        load_dataset_not_first_time()

def classify(root, chosen_algo, X_data, y_data, show_classification):
    show_classification.destroy()
    global load_dataset
    #load_dataset.destroy()
    
    global classification_count
    classification_count = classification_count + 1
    def fun():        
        featureList=[]
        s = deque(stack)

        for i in range(len(stack)):
            si = s.popleft() 
            val = si.get("1.0","end-1c")
            featureList.append(float(val))
        print(featureList)
        stack.clear()

        if chosen_algo == "Naive Bayes Classifier":
            GUI_msg, predicted_class = Naive_Bayes_Classifier(featureList, X_data, y_data)
            result_label = Label(root, text = GUI_msg, font='Arial 18 bold', fg='white', 
                          bg="grey")
            result_label.place(x=100, y=350)
            result_label.after(8, result_label.pack_forget())

        elif chosen_algo == "k-Nearest Neighbor":
            GUI_msg, predicted_class = k_Nearest_Neighbor(featureList, X_data, y_data)
            result_label = Label(root, text = GUI_msg, font='Arial 18 bold', fg='white', 
                          bg="grey")
            result_label.place(x=100, y=350)
            result_label.after(8, result_label.pack_forget())
            
        elif chosen_algo == "Decision Tree":
            GUI_msg, predicted_class = Decision_Tree_Classifier(featureList, X_data, y_data)
            result_label = Label(root, text = GUI_msg, font='Arial 18 bold', fg='white', 
                          bg="grey")
            result_label.place(x=100, y=350)
            result_label.after(8, result_label.pack_forget())
        #root.after(10000, lambda:root.destroy())
    if classification_count > 1:
        # clear window root
        lst = root.pack_slaves()
        for l in lst:
            l.destroy()
        canvas = tk.Canvas(root, height=500, width=1100, bg='grey')
        canvas.pack()
        
        no_of_features = len(X_data.columns)
        input_label = Label(root, 
                      text = "Enter Unseen sample in below "+str(no_of_features)+" entry cells and then click on "
                      "Show Classification button",
                      font='Arial 16 bold', fg='white', bg="grey").place(x=100, y=100)
        horizontal=105
        hori_ratio = int((900/no_of_features))
        width_ratio = int(hori_ratio/(no_of_features**2))

        for i in range(no_of_features):
            value = i#'f%d'%(i+1)
            if width_ratio<10:
                width_ratio = 10
            t= Text(root, height=1, width=width_ratio, font='Arial '+str(width_ratio))
            #t.insert('1.0', value)
            t.place(x=horizontal, y=150)
            stack.append(t)
            horizontal = horizontal + hori_ratio

        show_classification = tk.Button(root, text='Show Classification', padx=35, pady=10,
                                        fg="white", bg="grey", command=fun)
            
        show_classification.pack(side=tk.RIGHT)    
    
    if classification_count==1:
        fun()
    
root = tk.Tk()
root.title('Classification System')
root.iconbitmap('class.ico')
root.resizable(False, False)
tit = tk.Label(root, text="Classification System", padx=25, pady=6, font=("", 12)).pack()
canvas = tk.Canvas(root, height=500, width=1100, bg='grey')
canvas.pack()
load_dataset = tk.Button(root, text='Load Dataset',
                        padx=35, pady=10,
                        fg="white", bg="grey", command=load_dataset)
load_dataset.pack(side=tk.LEFT)
"""show_classification = tk.Button(root, text='Show Classification',
                        padx=35, pady=10,
                        fg="white", bg="grey", command=classify)

show_classification.pack(side=tk.RIGHT)"""
root.mainloop()